<a href="https://colab.research.google.com/github/Prabhu-shukla/Prabhu-shukla/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# we are developing a Random forest model to predict the satisfaction level of
# pasenger with facilities provided by the airline

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import pickle as pkl



In [4]:
# First of all we will use EDA to make the data "sensible" (LOL)


In [8]:
air_data = pd.read_csv("airlines_data.csv")
air_data.head()

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0


In [10]:
air_data.dtypes

,0
satisfaction,object
Customer Type,object
Age,int64
Type of Travel,object
Class,object
Flight Distance,int64
Seat comfort,int64
Departure/Arrival time convenient,int64
Food and drink,int64
Gate location,int64


In [12]:
air_data.shape

(129880, 22)

In [ ]:
# we will chech the missing vales

In [15]:
air_data.isna().any(axis=1).sum()

np.int64(393)

In [27]:
air_data_clean = air_data.dropna()
air_data_clean.isna().sum()

,0
satisfaction,0
Customer Type,0
Age,0
Type of Travel,0
Class,0
Flight Distance,0
Seat comfort,0
Departure/Arrival time convenient,0
Food and drink,0
Gate location,0


In [24]:
# now we dont have any missing values

In [25]:
# we will use ONE-HOT coding to convert categorical data into the numerical data

In [28]:
air_data_clean = pd.get_dummies(air_data_clean, columns= ['Customer Type', 'Class', 'Type of Travel'] )
air_data_clean

,satisfaction,Age,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Class_Business,Class_Eco,Class_Eco Plus,Type of Travel_Business travel,Type of Travel_Personal Travel
0,satisfied,65,265,0,0,0,2,2,4,2,3,3,0,3,5,3,2,0,0.0,True,False,False,True,False,False,True
1,satisfied,47,2464,0,0,0,3,0,2,2,3,4,4,4,2,3,2,310,305.0,True,False,True,False,False,False,True
2,satisfied,15,2138,0,0,0,3,2,0,2,2,3,3,4,4,4,2,0,0.0,True,False,False,True,False,False,True
3,satisfied,60,623,0,0,0,3,3,4,3,1,1,0,1,4,1,3,0,0.0,True,False,False,True,False,False,True
4,satisfied,70,354,0,0,0,3,4,3,4,2,2,0,2,4,2,5,0,0.0,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129875,satisfied,29,1731,5,5,5,3,2,5,2,2,3,3,4,4,4,2,0,0.0,False,True,False,True,False,False,True
129876,dissatisfied,63,2087,2,3,2,4,2,1,1,3,2,3,3,1,2,1,174,172.0,False,True,True,False,False,False,True
129877,dissatisfied,69,2320,3,0,3,3,3,2,2,4,4,3,4,2,3,2,155,163.0,False,True,False,True,False,False,True
129878,dissatisfied,66,2450,3,2,3,2,3,2,2,3,3,2,3,2,1,2,193,205.0,False,True,False,True,False,False,True


In [29]:
air_data_clean.dtypes

,0
satisfaction,object
Age,int64
Flight Distance,int64
Seat comfort,int64
Departure/Arrival time convenient,int64
Food and drink,int64
Gate location,int64
Inflight wifi service,int64
Inflight entertainment,int64
Online support,int64


In [30]:
# Afte the data has been clened and the necessary transformation has been done
# we can start model development

In [32]:
# we will seperate the data into target variable and predictor variable
X = air_data_clean.drop(columns=['satisfaction'])
y = air_data_clean['satisfaction']

In [33]:
# next step is to split the data into training, testing and validation set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
x_tr, x_val, y_tr, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

In [36]:
# before tuning the model we will set the hyper parameters
cv_params = {'n_estimators': [30,100],
              'max_depth' : [10,50],
               'min_samples_leaf' : [0.5 , 1],
               'min_samples_split' : [0.001,0.01],
               'max_features' : ['sqrt'],
               'max_samples' : [0.5, 0,9]}


In [38]:
# now we will create split index
split_index = [-1 if x in x_val.index else 0 for x in x_train.index]
custom_split = PredefinedSplit(test_fold=split_index)

In [39]:
# Instantiation of the model
rf = RandomForestClassifier(random_state=42)

In [40]:
rf_val = GridSearchCV(estimator=rf, param_grid=cv_params, cv=custom_split, verbose=1, n_jobs=-1)

In [42]:
%%time
rf_val.fit(x_train, y_train)

Fitting 1 folds for each of 48 candidates, totalling 48 fits
CPU times: user 19.7 s, sys: 362 ms, total: 20 s
Wall time: 38.6 s


GridSearchCV(cv=PredefinedSplit(test_fold=array([ 0,  0, ...,  0, -1])),
             estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [10, 50], 'max_features': ['sqrt'],
                         'max_samples': [0.5, 0, 9],
                         'min_samples_leaf': [0.5, 1],
                         'min_samples_split': [0.001, 0.01],
                         'n_estimators': [30, 100]},
             verbose=1)

In [43]:
rf_val.best_params_

{'max_depth': 50,
 'max_features': 'sqrt',
 'max_samples': 0.5,
 'min_samples_leaf': 1,
 'min_samples_split': 0.001,
 'n_estimators': 100}

In [44]:
# these are the best parameters to devel the optimal model

In [47]:
rf_opt= RandomForestClassifier(n_estimators= 100,
                               max_depth= 50,
                               max_features= 'sqrt',
                               max_samples= 0.5,
                               min_samples_leaf= 1,
                               min_samples_split= 0.01,
                               random_state=0)

In [48]:
# now we will fit the optimal model
rf_opt.fit(x_train, y_train)

RandomForestClassifier(max_depth=50, max_samples=0.5, min_samples_split=0.01,
                       random_state=0)

In [49]:
# Prediction on the test data by this optimal model
y_pred = rf_opt.predict(x_test)

In [51]:
# next we will see the precision score
pc_score= precision_score(y_test, y_pred, pos_label = "satisfied" )
pc_score


0.9032240286580325

In [52]:
# recall score
rc_score = recall_score(y_test, y_pred, pos_label = "satisfied")
rc_score

0.9271894093686355

In [54]:
# Accuracy
acc_score = accuracy_score(y_test, y_pred)
acc_score

0.9059990114914124

In [55]:
# F1 Score
fi_score = f1_score(y_test, y_pred, pos_label = "satisfied")
fi_score

0.9150498311046593